# L4: Optimize DSPy Agent with DSPy Optimizer

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"] = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [2]:
import mlflow

In [3]:
from helper import get_mlflow_tracking_uri

mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [4]:
mlflow.set_experiment("dspy_course_4")

2025/07/19 20:36:30 INFO mlflow.tracking.fluent: Experiment with name 'dspy_course_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/368932023392433418', creation_time=1752957390755, experiment_id='368932023392433418', last_update_time=1752957390755, lifecycle_stage='active', name='dspy_course_4', tags={}>

In [5]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [6]:
import dspy

dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Build a RAG Agent

In [7]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

In [8]:
import json

# Load trainset
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [9]:
# Overview of the dataset.
print(trainset[0])

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})


In [10]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [11]:
dspy.cache.load_memory_cache("./memory_cache.pkl")

In [12]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/07/19 20:36:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e705021164564f7584379f1da47cc1f0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/07/19 20:36:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/07/19 20:36:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/19 20:36:37 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/19 20:36:37 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 18%|█▊        | 18/100 [00:01<00:06, 13.57it/s]

Bootstrapped 4 full traces after 18 examples for up to 1 rounds, amounting to 18 attempts.


Bootstrapping set 4/6


...


🏃 View run eval_minibatch_19 at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418/runs/78beff4b559a412f8c13dd0b8b8e69bf
🧪 View experiment at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418
Average Metric: 54.00 / 100 (54.0%): 100%|██████████| 100/100 [00:04<00:00, 24.31it/s]

2025/07/19 20:37:29 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)
2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 54.0
2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [31.0, 50.0, 49.0, 49.0, 54.0]
2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 54.0
2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/19 20:37:30 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 54.0!



🏃 View run eval_full_4 at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418/runs/f08c17bf4d5741d991786684d7aa7e85
🧪 View experiment at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418


🏃 View run calm-goose-572 at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418/runs/e705021164564f7584379f1da47cc1f0
🧪 View experiment at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418


[Trace(request_id=6bb4d64987774ef099d055ab8f569f0a), Trace(request_id=828918bd9c9749a0836eafe2582a0763), Trace(request_id=e917067e4be4446b9c39c3b234ee59ec), Trace(request_id=fda8c0f9cbd940daa81bf6c334c52426), Trace(request_id=512f3fddbf274d2f9908368ca85468c0), Trace(request_id=4672fde81ccc4c8490caf890b07c0207), Trace(request_id=4062eee2c38b4298801f1de3836247fa), Trace(request_id=5607c9c1e1ff43c4b898baf6da02e3ed), Trace(request_id=2c468f2af03e45418ec08645539d5143), Trace(request_id=887facd8674244448800c9581d4a3f51)]

In [13]:
optimized_react.react.signature

StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia. It takes arguments {'query': {'type': 'string'}} in JSON format.\n(2) finish, whose description is <desc>Marks the task as complete. That is, signals

In [14]:
optimized_react.react.demos

[Example({'augmented': True, 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?', 'trajectory': '[[ ## thought_0 ## ]]\nI need to find out which studio produced both "That Darn Cat!" and "Never a Dull Moment." This information is likely available on Wikipedia, so I will search for it there.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "That Darn Cat! and Never a Dull Moment studio production"}\n\n[[ ## observation_0 ## ]]\n[1] «That Darn Cat! | That Darn Cat! is a 1965 American Walt Disney Productions thriller comedy film starring Hayley Mills (in her last of the six films she made for the Walt Disney Studios) and Dean Jones (starring in his first film for Disney) in a story about bank robbers, a kidnapping and a mischievous cat. The film was based on the 1963 novel "Undercover Cat" by Gordon and Mildred Gordon and was directed by Robert Stevenson. The title song was written by the Sherman Brothers and sung by Bo

In [15]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [16]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

Average Metric: 31.00 / 100 (31.0%): 100%|██████████| 100/100 [00:03<00:00, 29.30it/s]

2025/07/19 20:37:33 INFO dspy.evaluate.evaluate: Average Metric: 31 / 100 (31.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","Steve McQueen, known as ""the king of cool,"" starred in the movie ""...","The movie is ""The Great Escape.""",
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which individual, Robert Kardas...",Robert Kardashian's family is well-known for their reality TV show...,Robert Kardashian's family had their own reality TV show.,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...","I searched for information about the cast of the 1986 film ""Shadow...",There is no information available about a Russian ballerina in the...,
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...",Nehemiah appointed Amashsai to work at the temple in Jerusalem. Th...,"The meaning of the name of the man who appointed Amashsai, Nehemia...",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements or ...,To gain access to 173 countries and territories with an Austrian p...,"In addition to the Austrian passport, travelers may need to obtain...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The animated creatures that are the title characters of the film b...,The animated creatures that are the title characters of the film b...,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to verify the nationalities and contribution...,Both Dorothy Arzner and Richard Wallace were confirmed to be Ameri...,"No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run victorious-robin-491 at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418/runs/8ef1eb8a2a934292933a860114fccaf5
🧪 View experiment at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418
Original score: 31.0


[Trace(request_id=29cd15dedc8d443f8c9a9bd6465a7d28), Trace(request_id=02fb6a225b53479cbb4e69859f4c33c0), Trace(request_id=c8f6474fba5d43a5a5ad710f56ec1944), Trace(request_id=cb016d43d0f949f3a3378f79e25ab506), Trace(request_id=e54588f2dacc48b6a6e698d20eaa6ea0), Trace(request_id=76df57a396c74c84b7b9538a7b649f67), Trace(request_id=45f1e7abcc9e4a918d3aa907cbb8f0a1), Trace(request_id=aa439da991554991a0a2ce301f34b27c), Trace(request_id=284f19a0e4bd4f7ab4ec4f36371f47ef), Trace(request_id=1aa36bf64a8d4942abbe0efbf2887210)]

In [17]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")

Average Metric: 54.00 / 100 (54.0%): 100%|██████████| 100/100 [00:04<00:00, 21.24it/s]

2025/07/19 20:37:38 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...",I found that Bud Ekins was Steve McQueen's stunt double in the fil...,The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,{'thought_0': 'I need to find out which family had their own reali...,"The Kardashian family, associated with Robert Kardashian, has thei...",Robert Kardashian,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in ""Shadows in Paradi...","In my search for the cast of ""Shadows in Paradise,"" I found that t...",Sofya Skya,✔️ [True]
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...","Amashsai was appointed by Nehemiah, and the name Amasai, which is ...","""Burdensome""",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements are...,The search results indicate that Austrian citizens have visa-free ...,"A valid Austrian passport, and potentially a visa or health docume...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the release date of the first al...,I found that the American actress and singer-songwriter Katey Saga...,"April 19, 1994",✔️ [True]
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The question pertains to animated creatures that are the title cha...,"Fairies (specifically Puck, Titania, and Oberon)",
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,"{'thought_0': ""I need to determine if both Dorothy Arzner and Rich...","I found that Dorothy Arzner was an American film director, and Ric...","No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run debonair-pug-783 at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418/runs/c294bdb5a1fe4d9b932b8e571251515c
🧪 View experiment at: https://s172-29-90-91p8080.lab-aws-production.deeplearning.ai//#/experiments/368932023392433418
Optimized score: 54.0


[Trace(request_id=7a0cdbc7dd3842f8abbe4caeaa26f5c5), Trace(request_id=fee34d51e22e4518bc85ed85e75296dd), Trace(request_id=a6851acb2c8a49658598b1db09f55be9), Trace(request_id=774033e673b848be87839874921e80fd), Trace(request_id=1b0a1b07bf2d4e82ac2ea17cae7ea64b), Trace(request_id=dc77f3846bd44c8293ea8270f152fdfb), Trace(request_id=e2881f6bcb46477ca21a677472f63f92), Trace(request_id=ddf78bb4a7244b4f92d136cc89cf1ed6), Trace(request_id=4161f58a94e74ff6b179769159437112), Trace(request_id=411ac8833f8c46798ce5964d723e0e8d)]